<a href="https://www.kaggle.com/code/hydramst/water-pollution-lr?scriptVersionId=98010586" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_validate
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/water-pollution/Submission.csv
/kaggle/input/water-pollution/Target.csv
/kaggle/input/water-pollution/Train.csv
/kaggle/input/water-pollution/Test.csv


In [170]:
DF_train = pd.read_csv('/kaggle/input/water-pollution/Train.csv', delimiter = ',')
DF_test = pd.read_csv('/kaggle/input/water-pollution/Test.csv', delimiter = ',')
Target = pd.read_csv('/kaggle/input/water-pollution/Target.csv', delimiter = ',')
Submission = pd.read_csv('/kaggle/input/water-pollution/Submission.csv', delimiter = ',')

# Check if ID's in train presented in train

In [ ]:
len(set(DF_train['id'].values).intersection(set(DF_test['id'].values)))/len(set(DF_test['id'].values))

63% of test ID's presented in train

In [300]:
ids_intest =pd.DataFrame(DF_test['id'].values)


In [302]:
ids_intest = pd.DataFrame(DF_test['id'].values)
ids_intrain = pd.DataFrame(set(DF_train['id'].values).intersection(set(DF_test['id'].values)))

In [278]:
in_train = []
notin_train = []

for value in ids_intest.values:
    if (value in ids_intrain.values):
        in_train +=[value]
    else:
        notin_train +=[value]
        
notin_train = pd.DataFrame(notin_train, columns=['id'])
in_train = pd.DataFrame(in_train, columns=['id'])

print('in train \n len = ',len(in_train))

print( 'not in train \n len = ', len(notin_train))

in train 
 len =  78
not in train 
 len =  44


In [291]:
notin_train = notin_train.values.tolist()

In [292]:
notin_train

[['FRHR80'],
 ['FRHR79'],
 ['FRDR509C'],
 ['FRCR457'],
 ['FRFR137'],
 ['ES020MSPF000000513'],
 ['FRCR90'],
 ['FRGR0121'],
 ['ES018MSPFES118MAR000480'],
 ['ES10574'],
 ['ES020MSPF000000382'],
 ['FRAR55'],
 ['FRER68A'],
 ['FRAR26'],
 ['FRAR06'],
 ['UKSC020451'],
 ['ES050MSPF011002003'],
 ['UKSC004206'],
 ['UKGB104026067040'],
 ['UKSC023264'],
 ['LT130107103'],
 ['UKSC005203'],
 ['FRGR1518'],
 ['SE642520-139531'],
 ['UKSC006832'],
 ['FRHR134'],
 ['ES030MSPF0808010'],
 ['ES030MSPF0919010'],
 ['ES030MSPF0316011'],
 ['ES030MSPF0521010'],
 ['BEVL11_181'],
 ['FRGR0150B'],
 ['FRDR114A'],
 ['BG2KA130R002'],
 ['ES020MSPF000000569'],
 ['UKGB104027064251'],
 ['DERW_DEBW_12-01'],
 ['DERW_DEBW_45-03'],
 ['FRGR0252'],
 ['FRHR173'],
 ['FRFR71A'],
 ['ES100MSPF2000150'],
 ['FI22_001_001'],
 ['FRGR0366B']]

### Concatinate train and test

In [265]:
df = pd.concat([DF_train, DF_test])

In [268]:
#df_newids = pd.DataFrame(pd.concat([notin_train['id'], df['id']], axis=0, ignore_index=True), columns=['id'])
#df_newids.to_dict

<bound method DataFrame.to_dict of               id
0         FRHR80
1         FRHR79
2       FRDR509C
3        FRCR457
4        FRFR137
...          ...
2479  UKSC010040
2480    FRHR216C
2481     FRFR201
2482   FRGR0366B
2483     FRDR625

[2484 rows x 1 columns]>

In [267]:
df = df.drop(columns = ['Unnamed: 0'])

In [270]:
df = df.append({'id':df_newids}, ignore_index=True)
df

,code,year,period,id,Country,tourists,venue,rate,food,glass,metal,other,paper,plastic,leather,green_waste,waste_recycling
0,CAS_7723-14-0,2009.0,2009-01--2009-12,ES014MSPFES-014-NR-038-000-02-02,Spain,50941692.0,4.0,87.158924,49.0,8.0,3.0,14.0,15.0,9.0,0.0,0.0,16.84
1,CAS_7723-14-0,2009.0,2009-01--2009-12,FRGR0220,France,71176346.0,0.0,0.000000,32.0,10.0,3.0,26.0,20.0,9.0,0.0,0.0,22.26
2,CAS_14265-44-2,2009.0,2009-01--2009-12,FRGR1496,France,71176346.0,4.0,0.000000,32.0,10.0,3.0,26.0,20.0,9.0,0.0,0.0,22.26
3,CAS_7723-14-0,2009.0,2009-01--2009-12,FRDR174,France,71176346.0,0.0,0.000000,32.0,10.0,3.0,26.0,20.0,9.0,0.0,0.0,22.26
4,CAS_7723-14-0,2009.0,2009-01--2009-12,FRGR1022,France,71176346.0,0.0,0.000000,32.0,10.0,3.0,26.0,20.0,9.0,0.0,0.0,22.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2436,CAS_7723-14-0,2010.0,2010-01--2010-12,FRHR216C,France,71176346.0,0.0,0.000000,32.0,10.0,3.0,26.0,20.0,9.0,0.0,0.0,22.26
2437,CAS_14265-44-2,2010.0,2010-01--2010-12,FRFR201,France,71176346.0,0.0,0.000000,32.0,10.0,3.0,26.0,20.0,9.0,0.0,0.0,22.26
2438,CAS_7723-14-0,2010.0,2010-01--2010-12,FRGR0366B,France,71176346.0,0.0,0.000000,32.0,10.0,3.0,26.0,20.0,9.0,0.0,0.0,22.26
2439,CAS_14265-44-2,2010.0,2010-01--2010-12,FRDR625,France,71176346.0,0.0,0.000000,32.0,10.0,3.0,26.0,20.0,9.0,0.0,0.0,22.26


## View categorical and numerical columns

In [239]:
cat_columns = []
num_columns = []

for column_name in df.columns:
    if (df[column_name].dtypes == object):
        cat_columns +=[column_name]
    else:
        num_columns +=[column_name]

print('categorical columns:\t ',cat_columns, '\n len = ',len(cat_columns))

print('numerical columns:\t ',  num_columns, '\n len = ',len(num_columns))

categorical columns:	  ['Country', 'code', 'id', 'period'] 
 len =  4
numerical columns:	  ['food', 'glass', 'green_waste', 'leather', 'metal', 'other', 'paper', 'plastic', 'rate', 'tourists', 'venue', 'waste_recycling', 'year', 0] 
 len =  14


## Move 'year' to num columns 

In [228]:
cat_columns = ['code', 'period', 'id', 'Country', 'year']
num_columns = [ 'tourists', 'venue', 'rate', 'food', 'glass', 'metal', 'other', 'paper', 'plastic', 'leather', 'green_waste', 'waste_recycling'] 


In [232]:
df_filled = df.ffill().bfill()

In [229]:
df_filled = df.fillna(df[num_columns].mean())

In [233]:
df_filled.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4924 entries, 0 to 2483
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   code             4924 non-null   object 
 1   year             4924 non-null   float64
 2   period           4924 non-null   object 
 3   Country          4924 non-null   object 
 4   tourists         4924 non-null   float64
 5   venue            4924 non-null   float64
 6   rate             4924 non-null   float64
 7   food             4924 non-null   float64
 8   glass            4924 non-null   float64
 9   metal            4924 non-null   float64
 10  other            4924 non-null   float64
 11  paper            4924 non-null   float64
 12  plastic          4924 non-null   float64
 13  leather          4924 non-null   float64
 14  green_waste      4924 non-null   float64
 15  waste_recycling  4924 non-null   float64
 16  id               4924 non-null   object 
dtypes: float64(13)

In [236]:
mean_targ = Target['polution'].mean()
mean_targ

0.08285256728589442

## OHE 

In [ ]:
df_ohe = pd.get_dummies(df[cat_columns])
df_all = pd.concat([df_ohe, df[num_columns]], axis=1)

# Scaling

In [ ]:
# Creating StandardScaler Object
scaler = StandardScaler()
df_scale = pd.DataFrame(scaler.fit_transform(df_all[num_columns]))
df_all = pd.concat([df_ohe.reset_index(drop=True), df_scale.reset_index(drop=True)], axis=1)

## Split back to test and train 


In [ ]:
len_test = len(DF_test)
train = df_all.iloc[:-len_test]
test = df_all.iloc[-len_test:]
print(DF_train.shape[0], train.shape[0], DF_test.shape[0], test.shape[0])

# Log Scaling 

In [ ]:
#target_log = np.log(Target['polution'])
#target_log
target_orig = Target['polution']
len(pd.DataFrame(train))

# Traintest split

In [ ]:
X, y = pd.DataFrame(train).values, target_orig.values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape[0], X_test.shape[0], y_train.shape[0], y_test.shape[0]

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

# GridSearchCV

In [ ]:
### define model
model = Ridge()

alphas = np.array([1,0.1,0.01,0.001,0.0001,0.000001, 0.0000001])

# define parameters
param = {
    'alpha':alphas,
    'positive':[True, False],
    'copy_X':[True,False],
    'fit_intercept':[True,False]
    
}
#grid = GridSearchCV(model, param, scoring='r2', cv=2, n_jobs=-1)
#grid.fit(X_train, y_train)
#predictions = grid.predict(X_test)
#print('R2 : %.4f' %  r2_score(y_test,predictions))
#print('R2 : %.4f' % grid.best_score_)

# ElasticNetCV

In [ ]:
model_ElasticCV = ElasticNetCV(n_alphas=1, fit_intercept=True, max_iter=100000, l1_ratio=0.9, eps=0.0001, cv=5)

#model_ElasticCV.fit(X_train, y_train)

#y_predict_ElasticCV = model_ElasticCV.predict(X_test)

#print('R2 : %.4f' %  r2_score(y_test,y_predict_ElasticCV))

# ElasticNet

In [ ]:
model_Elastic = ElasticNet(alpha=0.00005, fit_intercept=True, max_iter=100000, l1_ratio=0.07)

model_Elastic.fit(X_train, y_train)

y_predict_Elastic = model_Elastic.predict(X_test)

print('R2 : %.4f' %  r2_score(y_test,y_predict_Elastic))

# Ridge LR

In [ ]:

model = Ridge(alpha = 0.1, fit_intercept=True, max_iter=10000)

model.fit(X_train, y_train)

y_predict=model.predict(X_test)

print('Error on test data:')
print('MSE: %.1f' % mse(y_test,y_predict))
print('RMSE: %.1f' % mse(y_test,y_predict,squared=False))
print('R2 : %.4f' %  r2_score(y_test,y_predict))

# Lasso

In [ ]:
model_L = Lasso(alpha=0.00001, fit_intercept=True)

model_L.fit(X_train, y_train)
y_predict=model_L.predict(X_test)

print('R2 : %.4f' %  r2_score(y_test,y_predict))

## Compare answers with original values

In [ ]:
def plotGraph(y_test,y_predict,regressorName):
    if max(y_test) >= max(y_predict):
        my_range = int(max(y_test))
    else:
        my_range = int(max(y_predict))
    plt.scatter(range(len(y_test)), y_test, color='blue')
    plt.scatter(range(len(y_predict)), y_predict, color='red')
    plt.title(regressorName)
    plt.show()
    return


plotGraph(y_test, y_predict_Elastic, "Elastic: RED - predicted. BLUE - original")
plotGraph(y_test, y_predict, "Ridge: RED - predicted. BLUE - original")

# Cross validation

In [ ]:
def cross_val(model, X_train, y_train):
  scoring = {'R2': 'r2',
            '-MSE': 'neg_mean_squared_error',
            '-MAE': 'neg_mean_absolute_error',
            'Max': 'max_error'}


  scores = cross_validate(model, X_train, y_train,
                        scoring=scoring, cv=ShuffleSplit(n_splits=5, random_state=42) )

  print('CV Results')
  DF_cv_linreg = pd.DataFrame(scores)
  display(DF_cv_linreg)
  print('\n')
  print(DF_cv_linreg.mean()[2:])
  print('\n')

cross_val(model_Elastic, X_train, y_train)

In [ ]:
model = Ridge(alpha=0.0001, max_iter=10000)

model.fit(X_train, y_train)

y_predict = model_Elastic.predict(test)

#y_predict = np.exp(y_predict)
Submission['polution'] = y_predict
Submission

In [ ]:
! rm /kaggle/working/water-pollution/Submission.csv
! mkdir /kaggle/working/water-pollution/
Submission['polution'][27] = 2.0

In [ ]:
Submission.to_csv('/kaggle/working/water-pollution/Submission.csv',index=False)

In [ ]:
! mkdir ~/.kaggle
! touch ~/.kaggle/kaggle.json
! echo '{"username":"hydramst","key": "" }' > ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
cat ~/.kaggle/kaggle.json

In [ ]:
#! kaggle competitions submit -f /kaggle/working/water-pollution/Submission.csv -m "Message" sf-matml-2022-regression